# extract pascal files  from label studio 
## put all image and annotations in single folder (Annotation,Image)

# 1. create label folder

In [1]:
import os
import xml.etree.ElementTree as ET

# Update this list with your actual class names
classes = [
    "Lane mark",
    "Plain road",
    "Manhole",
    "Divider",
    "Speed bumps",
    "Cracks",
    "Pothole",
    "Zebra crossing",
    "Footpath",
    "Rumble strips",
    "Sign board",
    "Leading lines"
]

# Folder paths
annotations_dir = 'E:\\JupyterNotebooks\\Road_problem\\yolo_pascal\\Annotation'
images_dir = 'E:\\JupyterNotebooks\Road_problem\\yolo_pascal\\Images'
output_labels_dir = 'E:\\JupyterNotebooks\\Road_problem\\yolo_pascal\\labels_yolo'

os.makedirs(output_labels_dir, exist_ok=True)

def convert(size, box):
    dw = 1.0 / size[0]
    dh = 1.0 / size[1]
    x = (box[0] + box[1]) / 2.0
    y = (box[2] + box[3]) / 2.0
    w = box[1] - box[0]
    h = box[3] - box[2]
    return (x * dw, y * dh, w * dw, h * dh)

for xml_file in os.listdir(annotations_dir):
    if not xml_file.endswith('.xml'):
        continue

    xml_path = os.path.join(annotations_dir, xml_file)
    tree = ET.parse(xml_path)
    root = tree.getroot()

    size = root.find('size')
    width = int(size.find('width').text)
    height = int(size.find('height').text)

    label_lines = []
    for obj in root.iter('object'):
        cls = obj.find('name').text
        if cls not in classes:
            continue
        cls_id = classes.index(cls)
        xmlbox = obj.find('bndbox')
        b = (
            float(xmlbox.find('xmin').text),
            float(xmlbox.find('xmax').text),
            float(xmlbox.find('ymin').text),
            float(xmlbox.find('ymax').text)
        )
        bb = convert((width, height), b)
        label_lines.append(f"{cls_id} {' '.join([str(round(a, 6)) for a in bb])}")

    # Save as .txt file in output_labels_dir
    txt_filename = os.path.splitext(xml_file)[0] + ".txt"
    txt_path = os.path.join(output_labels_dir, txt_filename)
    with open(txt_path, "w") as f:
        f.write("\n".join(label_lines))


# 2. Split Dataset

In [2]:
import os
import shutil
import random

# Paths to original images and labels
images_dir = r"E:\JupyterNotebooks\Road_problem\yolo_pascal\images"              # your images folder
labels_dir = r"E:\JupyterNotebooks\Road_problem\yolo_pascal\labels_yolo"  # your YOLO labels folder

# Output folders for split

output_dir = r"E:\JupyterNotebooks\Road_problem\pascal_yolo_dataset"
train_img_dir = os.path.join(output_dir, "images", "train")
val_img_dir = os.path.join(output_dir, "images", "val")
train_label_dir = os.path.join(output_dir, "labels", "train")
val_label_dir = os.path.join(output_dir, "labels", "val")

# Create directories if they don't exist
for folder in [train_img_dir, val_img_dir, train_label_dir, val_label_dir]:
    os.makedirs(folder, exist_ok=True)

# List all image files
image_files = [f for f in os.listdir(images_dir) if f.endswith(".jpg") or f.endswith(".png")]
random.shuffle(image_files)

# Split ratio
split_ratio = 0.8
split_index = int(len(image_files) * split_ratio)

train_files = image_files[:split_index]
val_files = image_files[split_index:]

def copy_files(file_list, img_dest, label_dest):
    for img_file in file_list:
        # Copy image
        src_img_path = os.path.join(images_dir, img_file)
        dst_img_path = os.path.join(img_dest, img_file)
        shutil.copy2(src_img_path, dst_img_path)

        # Copy corresponding label (same filename with .txt extension)
        label_file = os.path.splitext(img_file)[0] + ".txt"
        src_label_path = os.path.join(labels_dir, label_file)
        dst_label_path = os.path.join(label_dest, label_file)

        if os.path.exists(src_label_path):
            shutil.copy2(src_label_path, dst_label_path)
        else:
            print(f"Warning: Label file not found for image {img_file}")

# Copy train files
copy_files(train_files, train_img_dir, train_label_dir)

# Copy val files
copy_files(val_files, val_img_dir, val_label_dir)

print(f"Dataset split completed:")
print(f"Train images: {len(train_files)}")
print(f"Validation images: {len(val_files)}")


Dataset split completed:
Train images: 766
Validation images: 192


## Check all frames contain corresponding labels

In [3]:
import os

# Set your image and label folder paths
image_folder = r"E:/JupyterNotebooks/Road_problem/pascal_yolo_dataset/images/train"
label_folder = r"E:/JupyterNotebooks/Road_problem/pascal_yolo_dataset/labels/train"

# Get all image filenames (excluding extension)
image_files = [os.path.splitext(f)[0] for f in os.listdir(image_folder) if f.endswith(('.jpg', '.png'))]

# Get all label filenames (excluding extension)
label_files = [os.path.splitext(f)[0] for f in os.listdir(label_folder) if f.endswith('.txt')]

# Find images without corresponding label files
missing_labels = [f for f in image_files if f not in label_files]

# Output results
if missing_labels:
    print("Images without matching label files:")
    for name in missing_labels:
        print(name + ".jpg")
else:
    print("✅ All images have corresponding label files.")


✅ All images have corresponding label files.


# 3. train yolo model

In [2]:
pip install ultralytics


In [6]:
from ultralytics import YOLO

# Load a pre-trained YOLOv8 model (Nano for speed or Small for better accuracy)
model = YOLO('yolov8n.pt')  # or 'yolov8s.pt', 'yolov8m.pt'

# Train the model
model.train(
    data='E:\\JupyterNotebooks\\Road_problem\\pascal_yolo_dataset\\data.yaml',  # path to your YAML file
    epochs=50,
    imgsz=640,
    batch=8,
    name='road_features_yolo'
)



New https://pypi.org/project/ultralytics/8.3.149 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.146  Python-3.10.0 torch-2.7.0+cpu CPU (12th Gen Intel Core(TM) i5-12450HX)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=E:\JupyterNotebooks\Road_problem\pascal_yolo_dataset\data.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=road_featur

train: Scanning E:\JupyterNotebooks\Road_problem\pascal_yolo_dataset\labels\train... 406 images, 5 backgrounds, 2 corru

train: E:\JupyterNotebooks\Road_problem\pascal_yolo_dataset\images\train\4211cc2b-a8.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     1.1148]
train: E:\JupyterNotebooks\Road_problem\pascal_yolo_dataset\images\train\fbcda968-images_37.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     1.0437      1.1803]
train: New cache created: E:\JupyterNotebooks\Road_problem\pascal_yolo_dataset\labels\train.cache
val: Fast image access  (ping: 0.10.0 ms, read: 823.2455.0 MB/s, size: 274.8 KB)



C:\Users\Dinesh\AppData\Roaming\Python\Python310\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
val: Scanning E:\JupyterNotebooks\Road_problem\pascal_yolo_dataset\labels\val... 102 images, 4 backgrounds, 1 corrupt: 

val: E:\JupyterNotebooks\Road_problem\pascal_yolo_dataset\images\val\57dcb8a7-images_34.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     1.0601]
val: New cache created: E:\JupyterNotebooks\Road_problem\pascal_yolo_dataset\labels\val.cache


...

                   all        101        185      0.401      0.424      0.404       0.16



50 epochs completed in 3.684 hours.
Optimizer stripped from runs\detect\road_features_yolo2\weights\last.pt, 6.3MB
Optimizer stripped from runs\detect\road_features_yolo2\weights\best.pt, 6.3MB

Validating runs\detect\road_features_yolo2\weights\best.pt...
Ultralytics 8.3.146  Python-3.10.0 torch-2.7.0+cpu CPU (12th Gen Intel Core(TM) i5-12450HX)
Model summary (fused): 72 layers, 3,007,988 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 7/7 [00:16<0


                   all        101        185      0.462      0.441      0.399      0.173
             Lane mark         29         34      0.329      0.265      0.152      0.044
            Plain road         42         47      0.686      0.702      0.761      0.416
               Manhole          8         11      0.714      0.455      0.564       0.26
               Divider          9          9      0.273      0.556      0.511      0.153
           Speed bumps          7          9      0.463      0.333      0.332      0.068
                Cracks         15         23          0          0     0.0766     0.0254
               Pothole          8         15      0.304      0.407      0.253     0.0748
        Zebra crossing          3          3      0.247      0.667      0.297     0.0877
         Rumble strips          3          3      0.214      0.667      0.555      0.434
            Sign board         24         30      0.851        0.8      0.849      0.335
         Leading line

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([ 0,  1,  2,  3,  4,  5,  6,  7,  9, 10, 11])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x0000019027D011B0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,  

# 4. detect the video or images and count no. of times each event occurs

In [21]:
from ultralytics import YOLO
from collections import Counter

# Load your trained model (replace with your model path or a pretrained one)
model = YOLO(r'E:\JupyterNotebooks\Road_problem\runs\detect\road_features_yolo2\weights\best.pt')

# Path to your input video
video_path = r'E:\JupyterNotebooks\Road_problem\runs\detect\road_features_yolo_inference\a.avi'

# Run prediction on video
results = model.predict(source=video_path, stream=True)  # stream=True gives frame-by-frame results

# Create a counter for detected classes
event_counts = Counter()

# Your class names (make sure this matches your dataset!)
class_names = [
    'Lane mark', 'Plain road', 'Manhole', 'Divider', 'Speed bumps', 'Cracks',
    'Pothole', 'Zebra crossing', 'Footpath', 'Rumble strips', 'Sign board', 'Leading lines'
]

# Process results frame by frame
for result in results:
    # result.boxes.cls gives the class ids detected in the current frame
    detected_classes = result.boxes.cls.cpu().numpy().astype(int)
    # Count the occurrences of each detected class in the frame
    event_counts.update(detected_classes)

# Print counts with class names
print("Event counts in video:")
for class_id, count in event_counts.items():
    print(f"{class_names[class_id]}: {count}")



video 1/1 (frame 1/0) E:\JupyterNotebooks\Road_problem\runs\detect\road_features_yolo_inference\a.avi: 480x640 1 Divider, 144.3ms
video 1/1 (frame 2/0) E:\JupyterNotebooks\Road_problem\runs\detect\road_features_yolo_inference\a.avi: 480x640 1 Divider, 220.3ms
video 1/1 (frame 3/0) E:\JupyterNotebooks\Road_problem\runs\detect\road_features_yolo_inference\a.avi: 480x640 1 Divider, 270.4ms
video 1/1 (frame 4/0) E:\JupyterNotebooks\Road_problem\runs\detect\road_features_yolo_inference\a.avi: 480x640 1 Divider, 199.4ms
video 1/1 (frame 5/0) E:\JupyterNotebooks\Road_problem\runs\detect\road_features_yolo_inference\a.avi: 480x640 1 Divider, 257.9ms
video 1/1 (frame 6/0) E:\JupyterNotebooks\Road_problem\runs\detect\road_features_yolo_inference\a.avi: 480x640 1 Divider, 258.2ms
video 1/1 (frame 7/0) E:\JupyterNotebooks\Road_problem\runs\detect\road_features_yolo_inference\a.avi: 480x640 1 Divider, 250.0ms
video 1/1 (frame 8/0) E:\JupyterNotebooks\Road_problem\runs\detect\road_features_yolo_inf